# Autograd

In [2]:
import torch

In [3]:
x = torch.tensor([1.], requires_grad=True) # requires_grad : 미분할 것이라고 미리 표시해두는 것
# 그래서 tensor 값이 소수로 들어가야 한다!

In [4]:
x = torch.tensor([1.])
print(x.requires_grad)
x.requires_grad=True # 이후에 붙여줄 수도 있음
print(x.requires_grad)

False
True


## detach()와 torch.no_grad

.requires_grad = False 대신 사용 가능한 것들!
- detach() & torch.no_grad

In [28]:
x = torch.tensor([1.], requires_grad=True)
x = x,detach() # 아마 설치 버전이 달라 안되는 것 같다
print(x.requires_grad)

NameError: name 'detach' is not defined

In [5]:
x = torch.tensor([1.], requires_grad=True)
with torch.no_grad():
  print(x.requires_grad)
  y=x**2
  print(y) # grad_fn을 기억하지 않는다!
print(x.requires_grad)
y=x**2
print(y)

True
tensor([1.])
True
tensor([1.], grad_fn=<PowBackward0>)


detach() vs torch.no_grad
- detach() : "requires_grad = False"와 동일하게 아에 requires_grad를 떼버리는 것
- no_grad : 해당 순간에만 grad_fn을 기억하지 않도록 하는 것 > requires_grad는 여전히 True이다!

언제 사용하느냐?
1. transfer learning
- 기존의 weights를 유지하면서, 일부분을 학습시키고 싶을 때!
  - 만약 강아지 100종에 대해서 pretrained된 모델을 사용하고, 나는 10종에 대해서만 한정해서 분류하고 싶다!
  - 해당 가중치는 유지하되 (freeze), 마지막 분류 단계 부분만 재학습하고 싶다
  - 이때 x.requires_grad = False를 취해 해당 부분은 업데이트가 되지 않도록 막아두는 것
- 보통 detach() 사용
1. 모델 테스트 단계
- 불필요하게 메모리를 쓸 일이 없기 때문에 저장하지 않도록 하는 것
- 보통 no_grad 사용

## 미분

In [6]:
x = torch.tensor([1.], requires_grad=True)
y=x**2
print(y) # PowBackward0 추가되어 있음
print()

print(x.grad) # 미분한 값은 grad에 있다 !
y.backward() # y에 대해 x 미분
print(x.grad)

tensor([1.], grad_fn=<PowBackward0>)

None
tensor([2.])


In [7]:
x = torch.tensor([1.], requires_grad=True)
y=x.argmax()
print(y) # PowBackward0 추가 X > 미분이 불가능하므로!
y.backward() # 에러

tensor(0)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [8]:
x = torch.tensor([1.], requires_grad=True)
y=x**2
z = 3*y
print(z) # MulBackward0 추가되어 있음 > 연산방식에 대해 기억을 하고 있다
z.backward() # requires_grad를 찾아 미분을 한다
# 즉, y에는 requires_grad가 없고 x에만 있으므로, dz/dy >> dy/dx 순인 chain rule를 활용해서 dz/dx를 값을 얻을 수 있는 것
print(x.grad)

tensor([3.], grad_fn=<MulBackward0>)
tensor([6.])


grad_fn 추가

In [9]:
x = torch.tensor([1.], requires_grad=True)
y=(3*x**2 +1)*3
print(y)

tensor([12.], grad_fn=<MulBackward0>)


곱셈, 제곱셈, 더하기 연산 다 했지만, grad_fn이 기억하는 것은 마지막 연산 계산 > 모든 연산 계산 방식을 나타내지는 않는다

In [11]:
print(y.grad) # requires_grad가 없으므로, 값이 없다

None


/var/folders/yr/33_dxfw93nzc66j2xmmvkxnw0000gn/T/ipykernel_90017/1324337507.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525474122/work/build/aten/src/ATen/core/TensorBody.h:485.)
  print(y.grad)


In [23]:
x = torch.tensor([1.], requires_grad=True)
y=x**2
z = 3*y

y.retain_grad() # 중간 stem에 대해서 미분하겠다고 변경하는 것

z.backward() # z를 y에 대해서 미분하겠다
# y는 1**2로, 1 값을 가지고 있으니까 z 미분한 것에 대해 y 값을 넣어 y.grad에 저장!
print(y.grad)

tensor([3.])


y.requires_grad=True 는 불가능! 태초부터 시작해야만 (=tensor) requires_grad 가 가능하다
- 또한, z식을 작성하기 전 requires_grad=True를 해야만 backward()가 가능함!

In [22]:
y=torch.tensor([1.])
z = 3*y
y.requires_grad=True
print(y.requires_grad) # True이지만, 이미 z를 정의한 이후로 시도했으므로 에러 발생
z.backward()

True


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

## 편미분

In [25]:
x=torch.tensor([1.], requires_grad=True)
y=torch.tensor([1.], requires_grad=True)
z = 2*x**2 + y**2
z.backward()

# z에 대해서 각각 편미분하고, 대입한 값이 들어가 있다
print(x.grad)
print(y.grad)

tensor([4.])
tensor([2.])


In [26]:
x=torch.tensor([1., 2., 3.], requires_grad=True)
y=torch.sum(x**2) # x1**2 + x2**2 + x3**2
print(y)

y.backward()
print(x.grad) # 스칼라를 벡터로 미분한 것

tensor(14., grad_fn=<SumBackward0>)
tensor([2., 4., 6.])


## make_dot

In [31]:
!pip install torchviz
from torchviz import make_dot


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [34]:
x = torch.tensor([1.,2.], requires_grad=True)
make_dot((x**2+1)**2)

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

<img src="image/make_dot.png" width="150">

다음과 같이 연산 방식을 그래프로 그려준다